In [0]:
spark

In [0]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]

df = spark.createDataFrame(data = simpleData, schema = columns)

df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [0]:
from pyspark.sql.functions import col,when,upper,lower

In [0]:
df.withColumn('Flag',when(col('salary')>3500,1).otherwise(0)).show()

+-------------+----------+------+----+
|employee_name|department|salary|Flag|
+-------------+----------+------+----+
|        James|     Sales|  3000|   0|
|      Michael|     Sales|  4600|   1|
|       Robert|     Sales|  4100|   1|
|        Maria|   Finance|  3000|   0|
|        James|     Sales|  3000|   0|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|   1|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
|         Saif|     Sales|  4100|   1|
+-------------+----------+------+----+



In [0]:
df_spark1=df.withColumn('Flag',when((col('department')=='Sales') & (col('salary')>3000),1).otherwise(0))

In [0]:
df_spark1.show()

+-------------+----------+------+----+
|employee_name|department|salary|Flag|
+-------------+----------+------+----+
|        James|     Sales|  3000|   0|
|      Michael|     Sales|  4600|   1|
|       Robert|     Sales|  4100|   1|
|        Maria|   Finance|  3000|   0|
|        James|     Sales|  3000|   0|
|        Scott|   Finance|  3300|   0|
|          Jen|   Finance|  3900|   0|
|         Jeff| Marketing|  3000|   0|
|        Kumar| Marketing|  2000|   0|
|         Saif|     Sales|  4100|   1|
+-------------+----------+------+----+



In [0]:
from pyspark.sql.functions import udf

In [0]:
from pyspark.sql.types import IntegerType

In [0]:
@udf(returnType=IntegerType())
def update_salary(a,b):
    return a*b

In [0]:
df_spark1.withColumn('new_sal',update_salary(col('salary'),col('flag'))).show()

+-------------+----------+------+----+-------+
|employee_name|department|salary|Flag|new_sal|
+-------------+----------+------+----+-------+
|        James|     Sales|  3000|   0|      0|
|      Michael|     Sales|  4600|   1|   4600|
|       Robert|     Sales|  4100|   1|   4100|
|        Maria|   Finance|  3000|   0|      0|
|        James|     Sales|  3000|   0|      0|
|        Scott|   Finance|  3300|   0|      0|
|          Jen|   Finance|  3900|   0|      0|
|         Jeff| Marketing|  3000|   0|      0|
|        Kumar| Marketing|  2000|   0|      0|
|         Saif|     Sales|  4100|   1|   4100|
+-------------+----------+------+----+-------+



In [0]:
df_spark1.select('*',update_salary(col('salary'),col('flag')).alias('new_sal')).show()

+-------------+----------+------+----+-------+
|employee_name|department|salary|Flag|new_sal|
+-------------+----------+------+----+-------+
|        James|     Sales|  3000|   0|      0|
|      Michael|     Sales|  4600|   1|   4600|
|       Robert|     Sales|  4100|   1|   4100|
|        Maria|   Finance|  3000|   0|      0|
|        James|     Sales|  3000|   0|      0|
|        Scott|   Finance|  3300|   0|      0|
|          Jen|   Finance|  3900|   0|      0|
|         Jeff| Marketing|  3000|   0|      0|
|        Kumar| Marketing|  2000|   0|      0|
|         Saif|     Sales|  4100|   1|   4100|
+-------------+----------+------+----+-------+



In [0]:
df_spark1.createOrReplaceTempView('emp')

In [0]:
%sql
select * from emp;

employee_name,department,salary,Flag
James,Sales,3000,0
Michael,Sales,4600,1
Robert,Sales,4100,1
Maria,Finance,3000,0
James,Sales,3000,0
Scott,Finance,3300,0
Jen,Finance,3900,0
Jeff,Marketing,3000,0
Kumar,Marketing,2000,0
Saif,Sales,4100,1


In [0]:
spark.udf.register(name='Total_Salary',f=update_salary)

In [0]:
%sql
select *,Total_Salary(salary,flag) as tot_pay from emp;

employee_name,department,salary,Flag,tot_pay
James,Sales,3000,0,0
Michael,Sales,4600,1,4600
Robert,Sales,4100,1,4100
Maria,Finance,3000,0,0
James,Sales,3000,0,0
Scott,Finance,3300,0,0
Jen,Finance,3900,0,0
Jeff,Marketing,3000,0,0
Kumar,Marketing,2000,0,0
Saif,Sales,4100,1,4100


In [0]:
df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [0]:
from pyspark.sql import Row

In [0]:
from pyspark.sql.functions import col,count,min,max,avg,sum

In [0]:
# Define data for the DataFrame
data = [
    Row(name="Alice", dept="HR", sex="Female", salary=50000),
    Row(name="Bob", dept="Engineering", sex="Male", salary=60000),
    Row(name="Charlie", dept="HR", sex="Male", salary=55000),
    Row(name="Diana", dept="Marketing", sex="Female", salary=45000),
    Row(name="Eve", dept="Engineering", sex="Female", salary=70000),
    Row(name="Frank", dept="Marketing", sex="Male", salary=47000),
    Row(name="Grace", dept="Engineering", sex="Female", salary=65000),
    Row(name="Hank", dept="HR", sex="Male", salary=53000),
    Row(name="Ivy", dept="Marketing", sex="Female", salary=48000),
    Row(name="Jack", dept="Engineering", sex="Male", salary=62000)
]

# Create DataFrame
df_emp = spark.createDataFrame(data)

# Show DataFrame
df_emp.show()

+-------+-----------+------+------+
|   name|       dept|   sex|salary|
+-------+-----------+------+------+
|  Alice|         HR|Female| 50000|
|    Bob|Engineering|  Male| 60000|
|Charlie|         HR|  Male| 55000|
|  Diana|  Marketing|Female| 45000|
|    Eve|Engineering|Female| 70000|
|  Frank|  Marketing|  Male| 47000|
|  Grace|Engineering|Female| 65000|
|   Hank|         HR|  Male| 53000|
|    Ivy|  Marketing|Female| 48000|
|   Jack|Engineering|  Male| 62000|
+-------+-----------+------+------+



In [0]:
df_emp.groupBy('Dept').pivot('sex').count().show()

+-----------+------+----+
|       Dept|Female|Male|
+-----------+------+----+
|Engineering|     2|   2|
|         HR|     1|   2|
|  Marketing|     2|   1|
+-----------+------+----+



In [0]:
df_emp.groupBy('Dept').pivot('sex').sum('Salary').show()

+-----------+------+------+
|       Dept|Female|  Male|
+-----------+------+------+
|Engineering|135000|122000|
|         HR| 50000|108000|
|  Marketing| 93000| 47000|
+-----------+------+------+



In [0]:
from pyspark.sql.window import Window

In [0]:
from pyspark.sql.functions import rank,dense_rank,row_number

In [0]:
df_emp.withColumn('row_no',row_number().over(Window.partitionBy('Dept').orderBy('Salary'))).show()

+-------+-----------+------+------+------+
|   name|       dept|   sex|salary|row_no|
+-------+-----------+------+------+------+
|    Bob|Engineering|  Male| 60000|     1|
|   Jack|Engineering|  Male| 62000|     2|
|  Grace|Engineering|Female| 65000|     3|
|    Eve|Engineering|Female| 70000|     4|
|  Alice|         HR|Female| 50000|     1|
|   Hank|         HR|  Male| 53000|     2|
|Charlie|         HR|  Male| 55000|     3|
|  Diana|  Marketing|Female| 45000|     1|
|  Frank|  Marketing|  Male| 47000|     2|
|    Ivy|  Marketing|Female| 48000|     3|
+-------+-----------+------+------+------+



In [0]:
df_emp.withColumn('row_no',row_number().over(Window.partitionBy('Dept').orderBy('Salary'))).where(col('row_no')==2).show()

+-----+-----------+----+------+------+
| name|       dept| sex|salary|row_no|
+-----+-----------+----+------+------+
| Jack|Engineering|Male| 62000|     2|
| Hank|         HR|Male| 53000|     2|
|Frank|  Marketing|Male| 47000|     2|
+-----+-----------+----+------+------+



In [0]:
df_pivoted=df_emp.groupBy('Dept').pivot('sex').count()

In [0]:
df_pivoted.show()

+-----------+------+----+
|       Dept|Female|Male|
+-----------+------+----+
|Engineering|     2|   2|
|         HR|     1|   2|
|  Marketing|     2|   1|
+-----------+------+----+



In [0]:
from pyspark.sql.functions import expr

In [0]:
df_pivoted.select('Dept',expr("stack(2,'male',male,'female',female) as (gender,count)")).show()

+-----------+------+-----+
|       Dept|gender|count|
+-----------+------+-----+
|Engineering|  male|    2|
|Engineering|female|    2|
|         HR|  male|    2|
|         HR|female|    1|
|  Marketing|  male|    1|
|  Marketing|female|    2|
+-----------+------+-----+



## Fillna, na.fill

In [0]:
data = [
    Row(name="Alice", age=25, salary=50000),
    Row(name=None, age=30, salary=None),  # Null value for name and salary
    Row(name="Charlie", age=None, salary=55000),  # Null value for age
    Row(name="Diana", age=28, salary=None),  # Null value for salary
    Row(name=None, age=None, salary=None)  # Null value for all columns
]

# Create DataFrame
df_1 = spark.createDataFrame(data)

# Show DataFrame
df_1.show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|  Alice|  25| 50000|
|   null|  30|  null|
|Charlie|null| 55000|
|  Diana|  28|  null|
|   null|null|  null|
+-------+----+------+



In [0]:
df_1.fillna("unknown").show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|  Alice|  25| 50000|
|unknown|  30|  null|
|Charlie|null| 55000|
|  Diana|  28|  null|
|unknown|null|  null|
+-------+----+------+



In [0]:
df_1.fillna(100).show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  Alice| 25| 50000|
|   null| 30|   100|
|Charlie|100| 55000|
|  Diana| 28|   100|
|   null|100|   100|
+-------+---+------+



In [0]:
df_1.na.fill('unknown',['name']).show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|  Alice|  25| 50000|
|unknown|  30|  null|
|Charlie|null| 55000|
|  Diana|  28|  null|
|unknown|null|  null|
+-------+----+------+



In [0]:
df_1.fillna({
    'name':'unkown',
    'age':40,
    'salary':100000
}).show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  Alice| 25| 50000|
| unkown| 30|100000|
|Charlie| 40| 55000|
|  Diana| 28|100000|
| unkown| 40|100000|
+-------+---+------+



## Sample Function

In [0]:
df_range=spark.range(1,101)

In [0]:
display(df_range)

id
1
2
3
4
5
6
7
8
9
10


In [0]:
df_r1=df_range.sample(fraction=.1)

In [0]:
display(df_r1)

id
5
15
17
25
58
70
78
79
80
87
